### Import Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix



### Import Dataset

In [3]:
# Check the current working directory
%pwd

# Update the path to the CSV file
salary = pd.read_csv("./archive/Salaries.csv")


/var/folders/6h/k5xpmwhd71n8dl9df11lt9ph0000gn/T/ipykernel_93128/3255281011.py:5: DtypeWarning: Columns (3,4,5,6,12) have mixed types. Specify dtype option on import or set low_memory=False.
  salary = pd.read_csv("./archive/Salaries.csv")


### View the Information About Dataframe

In [4]:
salary.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148654 entries, 0 to 148653
Data columns (total 13 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Id                148654 non-null  int64  
 1   EmployeeName      148654 non-null  object 
 2   JobTitle          148654 non-null  object 
 3   BasePay           148049 non-null  object 
 4   OvertimePay       148654 non-null  object 
 5   OtherPay          148654 non-null  object 
 6   Benefits          112495 non-null  object 
 7   TotalPay          148654 non-null  float64
 8   TotalPayBenefits  148654 non-null  float64
 9   Year              148654 non-null  int64  
 10  Notes             0 non-null       float64
 11  Agency            148654 non-null  object 
 12  Status            38119 non-null   object 
dtypes: float64(3), int64(2), object(8)
memory usage: 14.7+ MB


In [5]:
salary.head()

,Id,EmployeeName,JobTitle,BasePay,OvertimePay,OtherPay,Benefits,TotalPay,TotalPayBenefits,Year,Notes,Agency,Status
0,1,NATHANIEL FORD,GENERAL MANAGER-METROPOLITAN TRANSIT AUTHORITY,167411.18,0.0,400184.25,NaN,567595.43,567595.43,2011,NaN,San Francisco,NaN
1,2,GARY JIMENEZ,CAPTAIN III (POLICE DEPARTMENT),155966.02,245131.88,137811.38,NaN,538909.28,538909.28,2011,NaN,San Francisco,NaN
2,3,ALBERT PARDINI,CAPTAIN III (POLICE DEPARTMENT),212739.13,106088.18,16452.6,NaN,335279.91,335279.91,2011,NaN,San Francisco,NaN
3,4,CHRISTOPHER CHONG,WIRE ROPE CABLE MAINTENANCE MECHANIC,77916.0,56120.71,198306.9,NaN,332343.61,332343.61,2011,NaN,San Francisco,NaN
4,5,PATRICK GARDNER,"DEPUTY CHIEF OF DEPARTMENT,(FIRE DEPARTMENT)",134401.6,9737.0,182234.59,NaN,326373.19,326373.19,2011,NaN,San Francisco,NaN


In [6]:
# get all possible values of each column if it is a string
for col in salary.columns:
    if salary[col].dtype == 'object':
        print(col, salary[col].unique())
        
  

EmployeeName ['NATHANIEL FORD' 'GARY JIMENEZ' 'ALBERT PARDINI' ... 'Charlene D Mccully'
 'Not provided' 'Joe Lopez']
JobTitle ['GENERAL MANAGER-METROPOLITAN TRANSIT AUTHORITY'
 'CAPTAIN III (POLICE DEPARTMENT)' 'WIRE ROPE CABLE MAINTENANCE MECHANIC'
 ... 'Conversion' 'Cashier 3' 'Not provided']
BasePay [167411.18 155966.02 212739.13 ... '6.04' '15.50' 'Not Provided']
OvertimePay [0.0 245131.88 106088.18 ... '102.70' '56.77' 'Not Provided']
OtherPay [400184.25 137811.38 16452.6 ... '13.35' 'Not Provided' '-618.13']
Benefits [nan 44430.12 69810.19 ... '0.13' '1.24' 'Not Provided']
Agency ['San Francisco']
Status [nan 'PT' 'FT']


In [ ]:
# find all the unique values in the JobTitle column
job_titles = salary['JobTitle'].unique()

# remove entries with 'Not provided' job title
salary = salary[salary['JobTitle'] != 'Not provided']

# sort the job titles alphabetically
job_titles.sort()

print(job_titles.size)

# remove leading and trailing whitespace
salary['JobTitle'] = salary['JobTitle'].str.strip()

salary['JobTitle'] = salary['JobTitle'].str.upper()

# fix capitalization of employee names
salary['EmployeeName'] = salary['EmployeeName'].str.title()

# convert BasePay, OvertimePay, OtherPay, Benefits, TotalPayBenefits to float
salary['BasePay'] = pd.to_numeric(salary['BasePay'], errors='coerce')
salary['OvertimePay'] = pd.to_numeric(salary['OvertimePay'], errors='coerce')
salary['OtherPay'] = pd.to_numeric(salary['OtherPay'], errors='coerce')
salary['Benefits'] = pd.to_numeric(salary['Benefits'], errors='coerce')
salary['TotalPayBenefits'] = pd.to_numeric(salary['TotalPayBenefits'], errors='coerce')

# Convert NaN in BasePay, OvertimePay, OtherPay, Benefits, TotalPayBenefits to 0
salary['BasePay'] = salary['BasePay'].fillna(0)
salary['OvertimePay'] = salary['OvertimePay'].fillna(0)
salary['OtherPay'] = salary['OtherPay'].fillna(0)
salary['Benefits'] = salary['Benefits'].fillna(0)
salary['TotalPayBenefits'] = salary['TotalPayBenefits'].fillna(0)

# find all negtive values in the TotalPayBenefits column
print(salary[salary['TotalPayBenefits'] < 0])



""" 
Negative income can have significant legal and financial implications for individuals and businesses. 
Individuals may have difficulty meeting financial obligations, accessing credit, or dealing with tax consequences. Businesses may face reporting requirements, 
tax implications, and bankruptcy considerations. It is important to address negative income promptly and seek professional advice to understand and address the 
underlying causes and consequences.
"""

salary

# * FT = Full Time, PT = Part Time

1636
            Id      EmployeeName                    JobTitle  BasePay  \
110528  110529  Timothy E Gibson            POLICE OFFICER 3      0.0   
110529  110530    Mark E Laherty            POLICE OFFICER 3      0.0   
110530  110531     David P Kucia            POLICE OFFICER 3      0.0   
148653  148654         Joe Lopez  COUNSELOR, LOG CABIN RANCH      0.0   

        OvertimePay  OtherPay  Benefits  TotalPay  TotalPayBenefits  Year  \
110528          0.0      0.00     -2.73      0.00             -2.73  2013   
110529          0.0      0.00     -8.20      0.00             -8.20  2013   
110530          0.0      0.00    -33.89      0.00            -33.89  2013   
148653          0.0   -618.13      0.00   -618.13           -618.13  2014   

        Notes         Agency Status  
110528    NaN  San Francisco    NaN  
110529    NaN  San Francisco    NaN  
110530    NaN  San Francisco    NaN  
148653    NaN  San Francisco     PT   (4, 13)


,Id,EmployeeName,JobTitle,BasePay,OvertimePay,OtherPay,Benefits,TotalPay,TotalPayBenefits,Year,Notes,Agency,Status
0,1,Nathaniel Ford,GENERAL MANAGER-METROPOLITAN TRANSIT AUTHORITY,167411.18,0.00,400184.25,0.0,567595.43,567595.43,2011,NaN,San Francisco,NaN
1,2,Gary Jimenez,CAPTAIN III (POLICE DEPARTMENT),155966.02,245131.88,137811.38,0.0,538909.28,538909.28,2011,NaN,San Francisco,NaN
2,3,Albert Pardini,CAPTAIN III (POLICE DEPARTMENT),212739.13,106088.18,16452.60,0.0,335279.91,335279.91,2011,NaN,San Francisco,NaN
3,4,Christopher Chong,WIRE ROPE CABLE MAINTENANCE MECHANIC,77916.00,56120.71,198306.90,0.0,332343.61,332343.61,2011,NaN,San Francisco,NaN
4,5,Patrick Gardner,"DEPUTY CHIEF OF DEPARTMENT,(FIRE DEPARTMENT)",134401.60,9737.00,182234.59,0.0,326373.19,326373.19,2011,NaN,San Francisco,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
148645,148646,Carolyn A Wilson,HUMAN SERVICES TECHNICIAN,0.00,0.00,0.00,0.0,0.00,0.00,2014,NaN,San Francisco,PT
148647,148648,Joann Anderson,COMMUNICATIONS DISPATCHER 2,0.00,0.00,0.00,0.0,0.00,0.00,2014,NaN,San Francisco,PT
148648,148649,Leon Walker,CUSTODIAN,0.00,0.00,0.00,0.0,0.00,0.00,2014,NaN,San Francisco,PT
148649,148650,Roy I Tillery,CUSTODIAN,0.00,0.00,0.00,0.0,0.00,0.00,2014,NaN,San Francisco,PT


In [52]:
print(job_titles)

['Account Clerk' 'Accountant' 'Accountant I' ... 'Youth Comm Advisor'
 'Youth Commission Advisor, Board Of Supervisors' 'Zoo Curator']


In [15]:
# How have salaries changed over time between different groups of people?
# Convert 'BasePay' to numeric, coercing errors to NaN
salary['BasePay'] = pd.to_numeric(salary['BasePay'], errors='coerce')

# group by year and title
grouped = salary.groupby(['JobTitle', 'Year'])
# find the mean of the base pay for each group
grouped = grouped['BasePay'].mean()

# print the grouped data
grouped.head()


    


JobTitle                     Year
ACCOUNT CLERK                2011    43300.806506
ACCOUNTANT                   2011    46643.172000
ACCOUNTANT INTERN            2011    28732.663958
ACPO,JuvP, Juv Prob (SFERS)  2014    62290.780000
ACUPUNCTURIST                2011    66374.400000
Name: BasePay, dtype: float64